https://adventofcode.com/2017/day/18

In [1]:
from collections import defaultdict, deque

In [2]:
with open("data/18.txt") as fh:
    data = fh.read()

In [3]:
def load_instructions(data):
    L = []
    for line in data.splitlines():
        inst = line.split()
        if len(inst) == 2:
            inst.append("")
        a, b, c = inst
        try:
            b = int(b)
        except ValueError:
            pass
        try:
            c = int(c)
        except ValueError:
            pass
        L.append((a, b, c))
    return L

In [4]:
instructions = load_instructions(data)
instructions

[('set', 'i', 31),
 ('set', 'a', 1),
 ('mul', 'p', 17),
 ('jgz', 'p', 'p'),
 ('mul', 'a', 2),
 ('add', 'i', -1),
 ('jgz', 'i', -2),
 ('add', 'a', -1),
 ('set', 'i', 127),
 ('set', 'p', 952),
 ('mul', 'p', 8505),
 ('mod', 'p', 'a'),
 ('mul', 'p', 129749),
 ('add', 'p', 12345),
 ('mod', 'p', 'a'),
 ('set', 'b', 'p'),
 ('mod', 'b', 10000),
 ('snd', 'b', ''),
 ('add', 'i', -1),
 ('jgz', 'i', -9),
 ('jgz', 'a', 3),
 ('rcv', 'b', ''),
 ('jgz', 'b', -1),
 ('set', 'f', 0),
 ('set', 'i', 126),
 ('rcv', 'a', ''),
 ('rcv', 'b', ''),
 ('set', 'p', 'a'),
 ('mul', 'p', -1),
 ('add', 'p', 'b'),
 ('jgz', 'p', 4),
 ('snd', 'a', ''),
 ('set', 'a', 'b'),
 ('jgz', 1, 3),
 ('snd', 'b', ''),
 ('set', 'f', 1),
 ('add', 'i', -1),
 ('jgz', 'i', -11),
 ('snd', 'a', ''),
 ('jgz', 'f', -16),
 ('jgz', 'a', -19)]

In [58]:
class Duet:
    def __init__(self, instructions):
        self.instructions = instructions
        self.registers = defaultdict(int)
        self.pos = 0
        self.last_sound_played = None
        self.funs = {
            "snd": self.snd,
            "set": self.set,
            "add": self.add,
            "mul": self.mul,
            "mod": self.mod,
            "rcv": self.rcv,
            "jgz": self.jgz
        }

    def run(self):
        while True:
            f, a, b = self.instructions[self.pos]
            try:
                self.funs[f]((f, a, b))
            except (IndexError, StopIteration):
                print("end")
                break

    def snd(self, args):
        _, a, _ = args
        self.last_sound_played = self.registers[a]
        self.pos += 1

    def set(self, args):
        _, a, b = args
        if isinstance(b, str):
            b = self.registers[b]
        self.registers[a] = b
        self.pos += 1

    def add(self, args):
        _, a, b = args
        if isinstance(b, str):
            b = self.registers[b]
        self.registers[a] += b
        self.pos += 1

    def mul(self, args):
        _, a, b = args
        if isinstance(b, str):
            b = self.registers[b]
        self.registers[a] *= b
        self.pos += 1

    def mod(self, args):
        _, a, b = args
        if isinstance(b, str):
            b = self.registers[b]
        self.registers[a] = self.registers[a] % b
        self.pos += 1

    def rcv(self, args):
        _, a, _ = args
        if self.registers[a]:
            print("Last sound played:", self.last_sound_played)
            raise StopIteration
        self.pos += 1

    def jgz(self, args):
        _, a, b = args
        if isinstance(a, str):
            a = self.registers[a]
        if isinstance(b, str):
            b = self.registers[b]
        if a > 0:
            self.pos += b
        else:
            self.pos += 1


In [59]:
test_instructions_txt = """\
set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2
"""
test_instructions = load_instructions(test_instructions_txt)

In [60]:
testduet = Duet(test_instructions)
testduet.run()

Last sound played: 4
end


In [62]:
duet = Duet(instructions)
duet.run()

Last sound played: 4601
end


In [69]:
class Duet2:
    def __init__(self, instructions):
        self.instructions = instructions
        self.registers = defaultdict(int)
        self.pos = 0
        self.other = None
        self.id = None
        self.queue = deque([])
        self.finished = False
        self.sends = 0
        self.funs = {
            "snd": self.snd,
            "set": self.set,
            "add": self.add,
            "mul": self.mul,
            "mod": self.mod,
            "rcv": self.rcv,
            "jgz": self.jgz
        }

    def run(self):
        if self.finished:
            print("finished", self.id)
            return
        try:
            while not self.finished:
                f, a, b = self.instructions[self.pos]
                self.funs[f]((f, a, b))
        except StopIteration:
            self.other.run()
        except IndexError:
            print("end", self.id)
            self.finished = True


    def snd(self, args):
        _, a, _ = args
        self.other.queue.append(self.registers[a])
        self.sends += 1
        self.pos += 1

    def rcv(self, args):
        _, a, _ = args
        if self.queue:
            self.registers[a] = self.queue.popleft()
            self.pos += 1
        else:
            if (not self.other.queue) and self.other.instructions[self.other.pos][0] == "rcv":
                print("deadlock", self.id)
                self.finished = True
                self.other.finished = True
                raise IndexError("deadlock")
            else:
                raise StopIteration

    def set(self, args):
        _, a, b = args
        if isinstance(b, str):
            b = self.registers[b]
        self.registers[a] = b
        self.pos += 1

    def add(self, args):
        _, a, b = args
        if isinstance(b, str):
            b = self.registers[b]
        self.registers[a] += b
        self.pos += 1

    def mul(self, args):
        _, a, b = args
        if isinstance(b, str):
            b = self.registers[b]
        self.registers[a] *= b
        self.pos += 1

    def mod(self, args):
        _, a, b = args
        if isinstance(b, str):
            b = self.registers[b]
        self.registers[a] = self.registers[a] % b
        self.pos += 1

    def jgz(self, args):
        _, a, b = args
        if isinstance(a, str):
            a = self.registers[a]
        if isinstance(b, str):
            b = self.registers[b]
        if a > 0:
            self.pos += b
        else:
            self.pos += 1


In [70]:
instructions = load_instructions(data)
d0 = Duet2(instructions)
d0.id = 0
d0.registers["p"] = 0

d1 = Duet2(instructions)
d1.id = 1
d1.registers["p"] = 1

d0.other = d1
d1.other = d0

d0.run()

deadlock 1
end 1


In [68]:
d1.sends

6858